In [1]:
# load data
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Model
from keras.layers import Flatten, Dense, Input
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Dropout
from keras.layers import GlobalMaxPooling2D, concatenate
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras import regularizers
import random

# get list of hold out data for ensemble training
random.seed(17332)
nums = random.sample(range(0,1507),60)

training_path = "../../Test_data/train.json"
testing_path = "../../Test_data/test.json"

train_data_whole = pd.read_json(training_path)
train_data = train_data_whole.loc[~train_data_whole.index.isin(list(nums))]

test_data = pd.read_json(testing_path)


# no third for inc angle
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))    # , c)))

    return np.array(imgs)

X = get_scaled_imgs(train_data)



Y = train_data["is_iceberg"]



X_full = X
Y_full = Y


X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.25,
                                                    random_state=42)

print("size x_train", len(X_train))
print("size x_test", len(X_test))

print("size y_train", len(y_train))
print("size y_test", len(y_test))

print("size x full", len(X_full))
print("size y full", len(Y_full))


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


size x_train 1158
size x_test 386
size y_train 1158
size y_test 386
size x full 1544
size y full 1544


In [2]:
# create the model
img_input = Input(shape=(75,75,3))
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv1')(img_input)
x = Convolution2D(64, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv1')(x)
x = Convolution2D(128, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block3_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
x = BatchNormalization()(x)

x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block4_conv1')(x)
x = Convolution2D(256, (3,3), activation='relu', padding='same',
                  kernel_regularizer=regularizers.l2(0.1),
                  bias_regularizer=regularizers.l2(0.1), name='block4_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
x = BatchNormalization()(x)

x = Flatten(name='flatten')(x)
x = Dense(512, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
x = Dense(256, kernel_regularizer=regularizers.l2(0.1),
          bias_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.5)(x)
main_output = Dense(1, activation='sigmoid', name='predictions')(x)

model = Model(inputs=[img_input], outputs= [main_output], name='vgg_inc')
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = "model_weights_basic_reg10_ensem.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=15)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 75, 75, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 75, 75, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 37, 37, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 37, 37, 64)        256       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 37, 37, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 37, 37, 128)       147584    
__________

In [3]:
# training code
#%% Image data augmentation 
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,                      # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,                  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,                 # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,                   # randomly flip images
    vertical_flip=False,                    # randomly flip images
    zoom_range=0.2)                     

datagen.fit(X_train)
model.fit_generator(datagen.flow(X_full, Y_full, shuffle=True,
                    batch_size=32), epochs = 150,
                     verbose= 1, callbacks=callbacks) # validation_data = (X_test, y_test),
                        #callbacks=callbacks)   # validation_data = (X_test, y_test)

Epoch 1/150
49/49 [==============================] - 13s 269ms/step - loss: 247.9362 - acc: 0.5542
Epoch 2/150
 1/49 [..............................] - ETA: 6s - loss: 247.1866 - acc: 0.5312

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:494: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:403: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


49/49 [==============================] - 7s 134ms/step - loss: 245.9437 - acc: 0.5917
Epoch 3/150
49/49 [==============================] - 7s 135ms/step - loss: 243.9568 - acc: 0.5976
Epoch 4/150
49/49 [==============================] - 7s 135ms/step - loss: 241.8573 - acc: 0.6377
Epoch 5/150
49/49 [==============================] - 7s 135ms/step - loss: 239.8573 - acc: 0.6353
Epoch 6/150
49/49 [==============================] - 7s 135ms/step - loss: 237.9020 - acc: 0.6575
Epoch 7/150
49/49 [==============================] - 7s 135ms/step - loss: 235.8486 - acc: 0.6785
Epoch 8/150
49/49 [==============================] - 7s 137ms/step - loss: 233.9592 - acc: 0.6627
Epoch 9/150
49/49 [==============================] - 7s 136ms/step - loss: 232.0210 - acc: 0.6761
Epoch 10/150
49/49 [==============================] - 7s 136ms/step - loss: 229.9876 - acc: 0.6932
Epoch 11/150
49/49 [==============================] - 7s 135ms/step - loss: 227.9783 - acc: 0.7015
Epoch 12/150
49/49 [==========

Epoch 85/150
49/49 [==============================] - 7s 133ms/step - loss: 97.5158 - acc: 0.8667
Epoch 86/150
49/49 [==============================] - 7s 133ms/step - loss: 96.0776 - acc: 0.8598
Epoch 87/150
49/49 [==============================] - 7s 134ms/step - loss: 94.6026 - acc: 0.8711
Epoch 88/150
49/49 [==============================] - 7s 134ms/step - loss: 93.1645 - acc: 0.8705
Epoch 89/150
49/49 [==============================] - 7s 134ms/step - loss: 91.7354 - acc: 0.8617
Epoch 90/150
49/49 [==============================] - 7s 133ms/step - loss: 90.3038 - acc: 0.8757
Epoch 91/150
49/49 [==============================] - 7s 134ms/step - loss: 88.8893 - acc: 0.8616
Epoch 92/150
49/49 [==============================] - 7s 133ms/step - loss: 87.4758 - acc: 0.8846
Epoch 93/150
49/49 [==============================] - 7s 133ms/step - loss: 86.1051 - acc: 0.8603
Epoch 94/150
49/49 [==============================] - 7s 134ms/step - loss: 84.7166 - acc: 0.8706
Epoch 95/150
49/49 [

In [4]:
# if doing no val set (i.e. final) only
filepath_full = 'model_weights_basic_reg10_ensem.hdf5'
model.save(filepath_full)

In [6]:
# from above, need to see the amount of epochs (80 +) that val stops increasing at, then do 
# that in one full go and save the model
filepath_full = 'model_weights_basic_reg10_ensem.hdf5'
from keras.models import load_model
import pandas as pd
# if not using val
file_path_use = filepath_full
# if using val
#file_path_use = file_path
inf_model = load_model(file_path_use)

score = inf_model.evaluate(X_full, Y_full, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


# for hold out submission to train ensemble
hold_out_train = train_data.iloc[nums]


X_sub = get_scaled_imgs(hold_out_train)

predicted_test=inf_model.predict(X_sub)

print("len of pred test", len(predicted_test))
print("len of id", len(test_data['id']))

submission_ensem = pd.DataFrame()
submission_ensem['id']=hold_out_train['id']
submission_ensem['is_iceberg']=predicted_test

submission_ensem.to_csv("sub_full_basic_ensem.csv", index=False)


1544/1544 [==============================] - 2s 2ms/step
Test loss: 28.1331595683
Test accuracy: 0.919041450777
len of pred test 60
len of id 8424


In [ ]:
# for test set submission
X_sub = get_scaled_imgs(test_data)

predicted_test=inf_model.predict(X_sub)

print("len of pred test", len(predicted_test))
print("len of id", len(test_data['id']))

submission = pd.DataFrame()
submission['id']=test_data['id']
submission['is_iceberg']=predicted_test



submission.to_csv('sub_full_basic_reg01_Fin.csv', index=False)